<a href="https://colab.research.google.com/github/PradeepChandrasekaran033/Data-Science-Projects-/blob/main/English_to_Japanese_model_translation_encoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U spacy
!python -m spacy download ja_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Embedding,LSTM,Dense,Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split as tts

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import spacy
nlp=spacy.load("ja_core_news_sm")

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
nplen=spacy.load("en_core_web_sm")

In [10]:
data=pd.read_csv("/content/drive/My Drive/JapaneseNLP.csv")

In [11]:
data.head()

,English,Japan
0,"Hello, how are you?",こんにちは元気ですか？
1,What is your name?,名前はなんですか？
2,Where are you from?,どこからきましたか？
3,Nice to meet you.,お会いできて嬉しいです。
4,How old are you?,何歳ですか？


In [ ]:
data.shape

(617, 2)

In [ ]:
data["English"][400]

"You can't always hold on to the things that are important. By letting them go, we gain something else."

In [ ]:
data["Japan"][400]

'大切なものを常に持ち続けることはできません。それらを手放すことで、私たちは何か他のものを得ることができます。'

In [ ]:
data["English"][100]

'The vibrant colors of the sunset painted the sky in hues of orange and pink.'

In [ ]:
data["Japan"][100]

'鮮やかな夕日が空をオレンジとピンクに染め上げました。'

In [ ]:
data.shape

(617, 2)

In [ ]:
data.isnull().any()

,0
English,False
Japan,False


In [ ]:
data.duplicated().sum()

np.int64(198)

In [12]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

(419, 2)

In [13]:
nlp=spacy.load("en_core_web_sm")
def eng_text(t):
    t=t.lower()
    t=nlp(t)
    w=[str(a) for a in t if not a.is_punct]
    return' '.join(w)

data["Eng_text"]=data["English"].apply(eng_text)
data.head()

,English,Japan,Eng_text
0,"Hello, how are you?",こんにちは元気ですか？,hello how are you
1,What is your name?,名前はなんですか？,what is your name
2,Where are you from?,どこからきましたか？,where are you from
3,Nice to meet you.,お会いできて嬉しいです。,nice to meet you
4,How old are you?,何歳ですか？,how old are you


In [14]:
def jap_text(t):
    t=t.lower()
    t=nlp(t)
    w=[str(a) for a in t if not a.is_punct]
    return' '.join(w)

data["Jap_text"]=data["Japan"].apply(jap_text)
data.head()

,English,Japan,Eng_text,Jap_text
0,"Hello, how are you?",こんにちは元気ですか？,hello how are you,こんにちは元気ですか
1,What is your name?,名前はなんですか？,what is your name,名前はなんですか
2,Where are you from?,どこからきましたか？,where are you from,どこからきましたか
3,Nice to meet you.,お会いできて嬉しいです。,nice to meet you,お会いできて嬉しいです
4,How old are you?,何歳ですか？,how old are you,何歳ですか


In [15]:
data.drop(["English","Japan"],axis=1,inplace=True)

In [ ]:
data.head()

,Eng_text,Jap_text
0,hello how are you,こんにちは元気ですか
1,what is your name,名前はなんですか
2,where are you from,どこからきましたか
3,nice to meet you,お会いできて嬉しいです
4,how old are you,何歳ですか


In [16]:
data["Jap_text"]=data["Jap_text"].apply(lambda x: f"<start> {x} <end>")

In [ ]:
data.head()

,Eng_text,Jap_text
0,hello how are you,<start> こんにちは元気ですか <end>
1,what is your name,<start> 名前はなんですか <end>
2,where are you from,<start> どこからきましたか <end>
3,nice to meet you,<start> お会いできて嬉しいです <end>
4,how old are you,<start> 何歳ですか <end>


In [17]:
train,test=tts(data,test_size=0.2,random_state=42)
train.head()

,Eng_text,Jap_text
66,have a nice day,<start> 良い１日を <end>
152,i 'm inspired by individuals who embrace their...,<start> 私は、自分の真正性を受け入れ、他の人にも同じようにするよう奨励する個人にイン...
31,i 'm happy,<start> 私は満足しています <end>
84,the cat is sleeping on the mat completely unaw...,<start> 猫はマットの上で寝ていて、周囲のことをまったく知りません <end>
472,you do n't have to see the whole staircase jus...,<start> 階段全体を見る必要はありません。最初の一歩を踏み出すだけです <end>


In [18]:
tok=Tokenizer(num_words=1000, oov_token="<00V>")
tok.fit_on_texts (train["Eng_text"])
seq=tok.texts_to_sequences (train["Eng_text"])
eng_seq=pad_sequences (seq, maxlen=45, padding="post")
eng_seq

array([[ 28,   7, 286, ...,   0,   0,   0],
       [  5,  11,  39, ...,   0,   0,   0],
       [  5,  11, 450, ...,   0,   0,   0],
       ...,
       [ 47,  27,  88, ...,   0,   0,   0],
       [ 12,  15,  20, ...,   0,   0,   0],
       [  2,  41,   4, ...,   0,   0,   0]], dtype=int32)

In [19]:
tok1=Tokenizer(num_words=1000, oov_token="<00V>")
tok1.fit_on_texts (train["Jap_text"])
seq1=tok1.texts_to_sequences ([t.split("<end>")[0] for t in train["Jap_text"]])
jap_seq=pad_sequences (seq1, maxlen=7, padding="post")
jap_seq

array([[  2,   4,   0, ...,   0,   0,   0],
       [  2,   5,   0, ...,   0,   0,   0],
       [  2,   6,   0, ...,   0,   0,   0],
       ...,
       [  2, 346,   0, ...,   0,   0,   0],
       [  2, 347,   0, ...,   0,   0,   0],
       [  2, 348,   0, ...,   0,   0,   0]], dtype=int32)

In [20]:
seq2=tok1.texts_to_sequences ([t.split("<start>")[1] for t in train["Jap_text"]])
jap_seq2=pad_sequences (seq2, maxlen=7, padding="post")
jap_seq2

array([[  4,   3,   0, ...,   0,   0,   0],
       [  5,   3,   0, ...,   0,   0,   0],
       [  6,   3,   0, ...,   0,   0,   0],
       ...,
       [346,   3,   0, ...,   0,   0,   0],
       [347,   3,   0, ...,   0,   0,   0],
       [348,   3,   0, ...,   0,   0,   0]], dtype=int32)

In [21]:
encoder_inp=eng_seq
decoder_inp=jap_seq2

In [22]:
max_decoder_seq_length = max(len(seq) for seq in decoder_inp)
max_decoder_seq_length

7

In [23]:
max_encoder_seq_length=max(len(seq) for seq in encoder_inp)
max_encoder_seq_length

45

In [24]:
num_encoder_tokens = len(tok.word_index) + 1
num_encoder_tokens

1034

In [27]:
decoder_target = np.zeros((decoder_inp.shape[0], decoder_inp.shape[1], 1))
decoder_target[:, 0:-1, 0] = decoder_inp[:, 1:]

In [25]:
num_decoder_tokens = len(tok1.word_index) + 1

In [28]:
decoder_target.shape

(335, 7, 1)

In [ ]:
print(num_encoder_tokens,'\n',num_decoder_tokens,'\n',max_encoder_seq_length,'\n',max_decoder_seq_length)

1034 
 349 
 45 
 7


In [29]:
nlstm=256

In [32]:
encoder_inputs = Input(shape=(None,))
enc_inp_embedded = Embedding(input_dim=num_encoder_tokens, output_dim=nlstm)(encoder_inputs)
enc_lstm = LSTM(nlstm, return_state=True)
enc_op, eh, ec = enc_lstm(enc_inp_embedded)
enc_states = [eh, ec]

decoder_inputs = Input(shape=(None,))
dec_inp_embedded = Embedding(input_dim=num_decoder_tokens, output_dim=nlstm)(decoder_inputs)
dec_lstm = LSTM(nlstm, return_state=True, return_sequences=True)
dec_op, dh, dc = dec_lstm(dec_inp_embedded, initial_state=enc_states)

dec_dense = Dense(num_encoder_tokens, activation='softmax')
dec_out = dec_dense(dec_op)

model = Model([encoder_inputs, decoder_inputs], dec_out)

In [33]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.fit([encoder_inp,decoder_inp],decoder_target,batch_size=32,epochs=10,validation_split=0.2)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 369ms/step - accuracy: 0.9936 - loss: 0.0454 - val_accuracy: 0.9915 - val_loss: 0.0699
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.9965 - loss: 0.0280 - val_accuracy: 0.9915 - val_loss: 0.0735
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 465ms/step - accuracy: 0.9953 - loss: 0.0371 - val_accuracy: 0.9915 - val_loss: 0.0744
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 300ms/step - accuracy: 0.9909 - loss: 0.0615 - val_accuracy: 0.9915 - val_loss: 0.0776
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 285ms/step - accuracy: 0.9964 - loss: 0.0261 - val_accuracy: 0.9915 - val_loss: 0.0777
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 390ms/step - accuracy: 0.9948 - loss: 0.0349 - val_accuracy: 0.9915 - val_loss: 0.0786
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 309ms/step - accuracy: 0.9948 - loss: 0.0381 - val_accuracy: 0.9915 - val_loss: 0.0804
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 333ms/step - accuracy: 0.9922 - loss: 0.0491 - val_accuracy: 0.9915 - val_loss:

In [39]:
tokt=Tokenizer(num_words=1000, oov_token="<00V>")
tokt.fit_on_texts (test["Eng_text"])
seqt=tokt.texts_to_sequences (test["Eng_text"])
engt_seq=pad_sequences (seqt, maxlen=45, padding="post")
engt_seq.shape

(84, 45)

In [40]:
toktj=Tokenizer(num_words=1000, oov_token="<00V>")
toktj.fit_on_texts (test["Jap_text"])
seqtj=toktj.texts_to_sequences ([t.split("<end>")[0] for t in test["Jap_text"]])
japt_seq=pad_sequences (seqtj, maxlen=7, padding="post")
japt_seq.shape

(84, 7)

In [41]:
decoder_test_target = np.zeros((japt_seq.shape[0], japt_seq.shape[1], 1))
decoder_test_target[:, 0:-1, 0] = japt_seq[:, 1:]
decoder_test_target.shape

(84, 7, 1)

In [42]:
loss,acc=model.evaluate([engt_seq,japt_seq],decoder_test_target)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.8521 - loss: 1.9031


In [43]:
acc*100

85.37414073944092